
# Web Search Agents — *Diabetes Treatment* 

This notebook is about **Diabetes treatment**, it includes:
- A **Search Agent** that runs a focused web search and produces a concise medical summary.
- A **Planner Agent** that generates multiple search queries covering distinct angles (drugs, devices, trials, guidelines, lifestyle).


## Environment Setup & Notes

- Loads environment variables for any search API keys if your `WebSearchTool` needs them.
- Uses your sample's API surface (`Agent`, `WebSearchTool`, `Runner`, `trace`, etc.).
- The **Search Agent** is concise by design (2–3 paragraphs, 300 words max).
- The **Planner Agent** returns a structured list of search queries via a Pydantic model.


In [17]:

# If you use .env for keys (optional)
from dotenv import load_dotenv
load_dotenv(override=True)


True


## 1) Imports

- `Agent`: defines a tool-using LLM agent
- `WebSearchTool`: executes web searches
- `Runner`: runs an agent with a message
- `trace`: optional tracing context manager for observability
- `ModelSettings`: configure tool choice behavior
- `BaseModel`/`Field`: define Pydantic schemas for planner output


In [18]:

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown



## 2) Search Agent (concise medical summary)

**Goal:** Given a diabetes-treatment query, search the web and produce a **concise 2–3 paragraph summary (≤300 words)**.  
**Scope:** Prioritize peer‑reviewed evidence, clinical guidelines (e.g., ADA), FDA approvals, pivotal trials, and technology (CGM, pumps, closed-loop).


In [19]:

SEARCH_INSTRUCTIONS = """
You are a medical research assistant. Given a search term about diabetes treatment,
search the web and produce a concise 2–3 paragraph summary (max 300 words).
Focus on: (1) new therapies and mechanisms, (2) clinical trial outcomes,
(3) FDA approvals and safety notes, and (4) current treatment guidelines.
Be specific and avoid speculation or marketing claims. Cite sources inline as [Site, Year] when available.
"""

search_agent = Agent(
    name="DiabetesTreatmentSearchAgent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],  # keep cost small; raise to "medium/high" if needed
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)



### Example — Quick Search
Change `message` to any query you want (e.g., *"GLP‑1/GIP updates 2025"*, *"Type 1 closed-loop systems 2025"*, *"ADA Standards of Care insulin titration"*).


In [20]:

import asyncio

# Example message
message = "Latest advancements in diabetes treatment 2025"

async def run_search():
    # Run with trace for observability (optional).
    with trace("DiabetesSearch"):
        result = await Runner.run(search_agent, message)
    # Display final condensed output
    display(Markdown(result.final_output))

# Execute inside the Jupyter event loop
await run_search()


As of August 2025, significant advancements in diabetes treatment have emerged across various domains, including novel therapies, technological innovations, and updated clinical guidelines.

**New Therapies and Mechanisms**

The FDA has approved Novo Nordisk's Ozempic (semaglutide) for reducing the risk of diabetic kidney disease progression, marking it as the first GLP-1 receptor agonist approved for this indication. Clinical trials demonstrated a 24% reduction in the risk of death from chronic kidney disease and major cardiovascular events. ([reuters.com](https://www.reuters.com/business/healthcare-pharmaceuticals/us-fda-approves-novo-nordisks-diabetes-drug-reduce-risk-worsening-kidney-disease-2025-01-28/?utm_source=openai))

In the realm of weight management, Eli Lilly's oral GLP-1 receptor agonist, orforglipron, has shown promising results in clinical trials, offering effective blood sugar control and weight loss comparable to injectable therapies, without serious side effects. If approved, it could be the first oral GLP-1 drug for weight loss, enhancing accessibility and affordability. ([time.com](https://time.com/7278429/weight-loss-obesity-pill-eli-lilly/?utm_source=openai))

**Clinical Trial Outcomes**

The REDEFINE 1 and REDEFINE 2 Phase III clinical trials evaluated the combination of cagrilintide and semaglutide in individuals with obesity and type 2 diabetes. Results indicated that the combination therapy led to a 13.7% weight loss over 68 weeks, compared to 3.1% with placebo, highlighting its potential as an effective weight management strategy. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Cagrilintide/semaglutide?utm_source=openai))

**FDA Approvals and Safety Notes**

In February 2025, the FDA approved insulin aspart-szjj, a biosimilar to Novolog, for medical use in the United States. This approval introduces a more affordable option for patients requiring rapid-acting insulin therapy. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Insulin_aspart?utm_source=openai))

**Current Treatment Guidelines**

The integration of artificial intelligence (AI) into continuous glucose monitoring (CGM) systems has been highlighted in recent conferences. AI-driven insights from CGMs have demonstrated significant improvements in glycemic control, suggesting a shift towards more personalized and efficient diabetes management strategies. ([drugtopics.com](https://www.drugtopics.com/view/ai-driven-cgm-insights-improved-glycemic-control-ada-2025?utm_source=openai))

These developments underscore a dynamic landscape in diabetes care, emphasizing personalized treatment approaches, technological integration, and innovative therapies to enhance patient outcomes.


## Recent Developments in Diabetes Treatment:
- [Canada approves Ozempic to reduce risk of diabetic kidney disease progression](https://www.reuters.com/business/healthcare-pharmaceuticals/canada-approves-ozempic-reduce-risk-diabetic-kidney-disease-progression-2025-08-19/?utm_source=openai)
- [Health Rounds: 3D printed insulin-producing cells show promise for type 1 diabetes in lab tests](https://www.reuters.com/business/healthcare-pharmaceuticals/health-rounds-3d-printed-insulin-producing-cells-show-promise-type-1-diabetes-2025-07-02/?utm_source=openai)
- [US FDA approves Novo Nordisk's Ozempic to cut risk of diabetic kidney disease progression](https://www.reuters.com/business/healthcare-pharmaceuticals/us-fda-approves-novo-nordisks-diabetes-drug-reduce-risk-worsening-kidney-disease-2025-01-28/?utm_source=openai) 


## 3) Planner Agent (multi-angle coverage)

**Goal:** Generate multiple complementary searches that cover different angles of **Diabetes treatment**.  
Use cases: seed a broader investigation, fan out to guidelines, drugs, devices, and lifestyle.


In [21]:

# Configurable number of distinct searches
HOW_MANY_SEARCHES = 4

PLANNER_INSTRUCTIONS = f"""
You are a helpful research planner. Given a query on diabetes treatment,
generate {HOW_MANY_SEARCHES} distinct web searches to cover the topic comprehensively.
Each search must focus on a different angle, choosing from:
- Pharmacotherapy (GLP-1, GIP, SGLT2, insulin analogs, fixed-ratio combos)
- Devices/technology (CGM, insulin pumps, closed-loop systems)
- Clinical guidelines & care standards (e.g., ADA Standards of Care 2025)
- Clinical trials/meta-analyses (efficacy, safety, outcomes)
- Lifestyle/behavioral interventions (nutrition, weight management, remission protocols)
- Special populations (pediatrics, pregnancy, CKD, CVD)
Return succinct reasons for each search.
"""

class WebSearchItem(BaseModel):
    reason: str = Field(description="Why this search matters for diabetes treatment")
    query: str = Field(description="Concrete search query to run")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description="List of complementary searches")
    
planner_agent = Agent(
    name="DiabetesTreatmentPlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)



### Example — Produce a Search Plan
Adjust `planner_message` to set your topic focus.


In [22]:

import asyncio

planner_message = "Diabetes treatment updates and standards of care"

async def run_plan():
    # Run planner (async)
    res = await Runner.run(planner_agent, planner_message)
    plan = res.final_output

    # Support either Pydantic model or plain dict output
    searches = getattr(plan, "searches", None)
    if searches is None and isinstance(plan, dict):
        searches = plan.get("searches", [])
    if searches is None:
        searches = []

    # Pretty print as Markdown
    plan_md = ["### Suggested Diabetes Treatment Searches", ""]
    for i, item in enumerate(searches, 1):
        query = getattr(item, "query", item.get("query") if isinstance(item, dict) else None)
        reason = getattr(item, "reason", item.get("reason") if isinstance(item, dict) else None)
        plan_md.append(f"**{i}.** `{query}`  \n*Why:* {reason}")
    display(Markdown("\n".join(plan_md)))

await run_plan()


### Suggested Diabetes Treatment Searches

**1.** `latest pharmacotherapy options for diabetes treatment 2023`  
*Why:* To understand the most recent pharmacological advancements in diabetes treatment, including new medications and updated guidelines.
**2.** `diabetes management technology updates 2023`  
*Why:* To explore how emerging technologies, such as continuous glucose monitors and insulin pumps, are improving diabetes management and patient outcomes.
**3.** `ADA Standards of Care diabetes 2025 updates`  
*Why:* To review the latest clinical guidelines for diabetes care, focusing on the ADA Standards of Care updates for 2025.
**4.** `recent clinical trials diabetes treatment safety efficacy 2023`  
*Why:* To examine recent findings from clinical trials and meta-analyses regarding the safety and efficacy of new diabetes treatments.


## 4) Run the Plan Automatically

This helper will take the **planner output**, run each search via the **Search Agent**, and concatenate concise summaries.


In [23]:
import asyncio

async def run_plan_and_summarize(plan: WebSearchPlan):
    """
    Run each planner query with search_agent and return a concatenated Markdown string.
    Accepts either a Pydantic object with .searches or a dict with ['searches'].
    """
    # Normalize searches
    searches = getattr(plan, "searches", None)
    if searches is None and isinstance(plan, dict):
        searches = plan.get("searches", [])
    if not searches:
        return "_No searches to run._"

    outputs = []
    for item in searches:
        query = getattr(item, "query", item.get("query") if isinstance(item, dict) else None)
        if not query:
            continue
        with trace(f"Search: {query}"):
            res = await Runner.run(search_agent, query)
        outputs.append(f"#### {query}\n\n{res.final_output}")
    return "\n\n---\n\n".join(outputs)

# Example: get a fresh plan and summarize (do NOT call run_plan(), it doesn't return a value)
plan_res = await Runner.run(planner_agent, planner_message)
combined = await run_plan_and_summarize(plan_res.final_output)
display(Markdown(combined))


#### latest updates GLP-1 GIP SGLT2 insulin analogs diabetes treatment

Recent advancements in diabetes treatment have introduced several innovative therapies targeting GLP-1, GIP, SGLT2, and insulin analogs. Tirzepatide, a dual GLP-1 and GIP receptor agonist, has demonstrated significant efficacy in managing type 2 diabetes. Clinical trials have shown that tirzepatide reduces HbA1c levels and body weight more effectively than semaglutide, a GLP-1 receptor agonist. Additionally, tirzepatide has been associated with a 38% reduction in major cardiovascular complications compared to placebo in individuals with obesity and heart failure with preserved ejection fraction. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Tirzepatide?utm_source=openai))

The American Diabetes Association (ADA) has updated its 2025 Standards of Care to emphasize the benefits of GLP-1 receptor agonists and SGLT2 inhibitors in patients with type 2 diabetes. The guidelines recommend these medications for individuals with established or high risk of atherosclerotic cardiovascular disease, heart failure, or chronic kidney disease. The ADA also highlights the importance of shared decision-making and individualized treatment plans. ([drugtopics.com](https://www.drugtopics.com/view/ada-updates-diabetes-treatment-guidelines-to-reelect-benefits-of-glp-1/american-pharmacists-month?utm_source=openai))

In the realm of insulin therapy, insulin icodec, a once-weekly basal insulin analog, has shown promise. A phase 3a trial demonstrated that insulin icodec provided a greater reduction in HbA1c levels compared to daily insulin glargine U100, with a similar safety profile. This long-acting insulin aims to improve patient adherence and glycemic control. ([onlinelibrary.wiley.com](https://onlinelibrary.wiley.com/doi/10.1002/kjm2.12800?utm_source=openai))

Combining GLP-1 receptor agonists with SGLT2 inhibitors has been associated with improved glycemic control, weight reduction, and cardiovascular benefits. A meta-analysis indicated that this combination therapy led to significant improvements in HbA1c levels, body weight, and systolic blood pressure compared to monotherapy. However, the combination may increase the risk of adverse effects such as vomiting and diarrhea. ([jabfm.org](https://www.jabfm.org/content/37/3/372?utm_source=openai))

These developments underscore the evolving landscape of diabetes treatment, focusing on personalized and combination therapies to enhance patient outcomes. 

---

#### diabetes treatment technology updates CGM insulin pumps 2023

In 2023, significant advancements in diabetes treatment technologies, particularly in continuous glucose monitoring (CGM) and insulin pumps, have enhanced the management of diabetes. The FDA cleared Abbott's FreeStyle Libre 2 and 3 sensors for integration with automated insulin delivery (AID) systems, allowing real-time insulin adjustments based on glucose data. These sensors are now approved for use by children as young as two years old and can be worn for up to 15 days. ([abbott.mediaroom.com](https://abbott.mediaroom.com/2023-03-06-U-S-FDA-Clears-Abbotts-FreeStyle-Libre-R-2-and-FreeStyle-Libre-R-3-Sensors-for-Integration-with-Automated-Insulin-Delivery-Systems?utm_source=openai))

Beta Bionics received FDA clearance for its iLet ACE Pump and iLet Dosing Decision Software in May 2023. This system, when paired with a compatible CGM, forms the iLet Bionic Pancreas, an automated insulin dosing system that utilizes an algorithm for personalized insulin delivery. The device is approved for individuals aged six and older with type 1 diabetes. ([techtarget.com](https://www.techtarget.com/pharmalifesciences/feature/FDA-approved-insulin-pumps-used-for-insulin-pump-therapy?utm_source=openai))

Roche's Accu-Chek Solo Micropump System, approved in August 2023, is a tubeless patch pump that integrates the insulin pump, infusion set, and reservoir into a single device, simplifying insulin therapy management. ([techtarget.com](https://www.techtarget.com/pharmalifesciences/feature/FDA-approved-insulin-pumps-used-for-insulin-pump-therapy?utm_source=openai))

Additionally, the FDA approved donislecel (Lantidra) in June 2023, a cellular therapy medication for adults with type 1 diabetes who experience severe hypoglycemia despite intensive management. Donislecel is an allogeneic pancreatic islet cellular therapy administered via intravenous infusion. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Donislecel?utm_source=openai))

These developments reflect a trend toward more integrated and personalized diabetes care, aiming to improve glycemic control and quality of life for individuals with diabetes. 

---

#### ADA Standards of Care 2025 diabetes treatment overview

The American Diabetes Association's (ADA) 2025 Standards of Care provide comprehensive, evidence-based guidelines for diagnosing and managing diabetes and prediabetes. These guidelines emphasize a person-centered approach, integrating the latest scientific research to enhance health outcomes.

**New Therapies and Mechanisms**

The 2025 Standards introduce several significant updates:

- **Continuous Glucose Monitoring (CGM) for Type 2 Diabetes**: For the first time, CGM is recommended for adults with type 2 diabetes, including those not on insulin therapy. This approach aims to improve glycemic control and reduce complications. ([diabetes.org](https://diabetes.org/newsroom/press-releases/american-diabetes-association-releases-standards-care-diabetes-2025?utm_source=openai))

- **GLP-1 Receptor Agonists**: The guidelines highlight the use of GLP-1 receptor agonists not only for weight management but also for their benefits in heart and kidney health. ([diabetes.org](https://diabetes.org/newsroom/press-releases/american-diabetes-association-releases-standards-care-diabetes-2025?utm_source=openai))

- **Weight Management Pharmacotherapy**: Recommendations now include the continuation of weight management medications beyond achieving weight loss goals to maintain health benefits and prevent weight regain. ([diabetesjournals.org](https://diabetesjournals.org/care/article/48/Supplement_1/S6/157564?utm_source=openai))

**Clinical Trial Outcomes**

The ADA's recommendations are grounded in recent clinical trials:

- **CGM in Type 2 Diabetes**: Studies have demonstrated that CGM use can reduce the risk of serious complications, including diabetic ketoacidosis, severe hypoglycemia, and hospitalizations for hyperglycemia. ([diatribe.org](https://diatribe.org/diabetes-management/your-guide-2025-ada-standards-care?utm_source=openai))

- **GLP-1 Receptor Agonists**: Clinical evidence supports the efficacy of GLP-1 receptor agonists in improving glycemic control and providing cardiovascular and renal benefits. ([diabetes.org](https://diabetes.org/newsroom/press-releases/american-diabetes-association-releases-standards-care-diabetes-2025?utm_source=openai))

**FDA Approvals and Safety Notes**

The 2025 Standards incorporate recent FDA approvals and safety considerations:

- **GLP-1 Receptor Agonists**: The FDA has approved several GLP-1 receptor agonists for weight management and cardiovascular risk reduction. ([diabetes.org](https://diabetes.org/newsroom/press-releases/american-diabetes-association-releases-standards-care-diabetes-2025?utm_source=openai))

- **Weight Management Pharmacotherapy**: The guidelines emphasize the importance of monitoring for side effects and provide guidance on the safe use of weight management medications. ([diabetesjournals.org](https://diabetesjournals.org/care/article/48/Supplement_1/S6/157564?utm_source=openai))

**Current Treatment Guidelines**

The ADA's 2025 Standards of Care outline the following key treatment strategies:

- **Person-Centered Care**: Treatment decisions should be timely, evidence-based, and made collaboratively with individuals, considering their preferences, prognoses, comorbidities, and financial considerations. ([guidelinecentral.com](https://www.guidelinecentral.com/guideline/14119/?utm_source=openai))

- **Integrated Care Models**: Approaches should align with evidence-based care models that emphasize team-based care, integrated long-term treatment of diabetes and comorbidities, and ongoing communication and goal setting. ([guidelinecentral.com](https://www.guidelinecentral.com/guideline/14119/?utm_source=openai))

- **Monitoring and Support**: Regular monitoring of obesity-related measurements and continued support for individuals who have achieved weight loss goals are crucial for maintaining health benefits. ([diabetesjournals.org](https://diabetesjournals.org/care/article/48/Supplement_1/S6/157564?utm_source=openai))

These guidelines aim to provide healthcare professionals with actionable strategies to improve the care and outcomes of individuals with diabetes. 

---

#### recent clinical trials diabetes treatment efficacy safety 2023

Recent clinical trials have evaluated the efficacy and safety of various diabetes treatments, leading to significant findings:

**Once-Weekly Insulin Regimens:** A systematic review and network meta-analysis published in January 2024 assessed the impact of once-weekly insulin regimens on glycemic control in type 2 diabetes patients. The study concluded that these regimens are effective in managing blood sugar levels, offering a potential alternative to daily insulin injections. However, the analysis also highlighted the need for careful monitoring of patients to ensure optimal outcomes. ([dmsjournal.biomedcentral.com](https://dmsjournal.biomedcentral.com/articles/10.1186/s13098-023-01240-5?utm_source=openai))

**SGLT-2 Inhibitors and Diabetic Ketoacidosis (DKA):** A systematic review and network meta-analysis published in 2023 examined the risk of DKA associated with SGLT-2 inhibitors in type 2 diabetes patients. The findings indicated that while these medications are generally safe, there is a slight increase in the risk of DKA, particularly in certain patient populations. The study emphasized the importance of patient selection and monitoring to mitigate this risk. ([frontiersin.org](https://www.frontiersin.org/journals/pharmacology/articles/10.3389/fphar.2023.1145587/full?utm_source=openai))

**GLP-1 Receptor Agonists in Prediabetes:** A systematic review and meta-analysis published in 2024 evaluated the efficacy and safety of GLP-1 receptor agonists in individuals with prediabetes. The study found that these medications effectively reduce the progression to type 2 diabetes and improve glycemic control. Safety profiles were favorable, with a low incidence of adverse effects. ([dmsjournal.biomedcentral.com](https://dmsjournal.biomedcentral.com/articles/10.1186/s13098-024-01371-3?utm_source=openai))

These findings underscore the ongoing advancements in diabetes treatment, highlighting the importance of personalized therapy and vigilant monitoring to optimize patient outcomes. 


## 5) Summary & Next Steps

- We have:
  1) A **Search Agent** that produces concise summaries of diabetes-treatment topics.
  2) A **Planner Agent** that proposes complementary searches to ensure coverage.
- Tweak knobs:
  - `search_context_size`: `"low"` → `"medium"`/`"high"` to broaden retrieval (higher cost).
  - `HOW_MANY_SEARCHES`: number of planner suggestions.
  - Instruction prompts for stricter tone (e.g., enforce citation style or word limits).

**Tip:** For production, consider adding:
- Caching of search results
- Source deduplication and quality filters (e.g., prioritize guideline bodies and peer‑reviewed journals)
- Safety checks for medical claims
- Cost/latency tracing and budgets



## 6) Email Agent (compose HTML & send)

This section imitates the pattern in your `lab4.ipynb`:
- A **tool** (`send_email`) that uses SendGrid to actually send mail.
- An **Email Agent** that takes a **markdown report** and outputs a subject + HTML body.
- A small **pipeline** that: *(plan → search → write report → compose email → send)*.

> Replace the `FROM_EMAIL` and `TO_EMAIL` with your own verified addresses.  
> Ensure `SENDGRID_API_KEY` is available in your environment (e.g., in `.env`).


In [24]:

import os
from typing import Dict
from pydantic import BaseModel, Field
from IPython.display import Markdown, display

# Tooling: SendGrid (optional; you can swap to your own email backend)
try:
    import sendgrid
    from sendgrid.helpers.mail import Mail, Email, To, Content
    _HAS_SENDGRID = True
except Exception as _e:
    _HAS_SENDGRID = False
    print("SendGrid not installed; email sending will be a no-op unless you install `sendgrid`.", _e)


FROM_EMAIL = os.environ.get("FROM_EMAIL", "wuguo5982.com")   # <--- change to your verified sender
TO_EMAIL = os.environ.get("TO_EMAIL", "wuguochen@gmail.com")       # <--- change to your recipient


In [25]:

# Register a function tool for sending email (matches your sample's style)
from agents import function_tool

@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send an email using SendGrid. Returns a status dict."""
    if not _HAS_SENDGRID:
        return {"status": "skipped", "reason": "sendgrid not installed"}
    api_key = os.environ.get("SENDGRID_API_KEY")
    if not api_key:
        return {"status": "skipped", "reason": "SENDGRID_API_KEY is missing"}
    sg = sendgrid.SendGridAPIClient(api_key=api_key)
    content = Content("text/html", html_body)
    mail = Mail(Email(FROM_EMAIL), To(TO_EMAIL), subject, content).get()
    resp = sg.client.mail.send.post(request_body=mail)
    return {"status": "success", "code": getattr(resp, 'status_code', None)}


In [26]:

# Email composer agent — converts Markdown to a professional HTML email + subject
from agents import Agent
from agents.model_settings import ModelSettings

EMAIL_INSTRUCTIONS = """
You are an email composition assistant. Given a Markdown report about diabetes treatment,
draft a clear, professional email:
- Write a concise, informative subject line
- Convert the Markdown into clean HTML
- Keep tone neutral, factual, and suitable for clinicians or analysts
Return your output strictly as a JSON object with keys:
  - subject (string)
  - html (string, HTML content)
No additional commentary.
"""

class EmailOutput(BaseModel):
    subject: str = Field(description="Email subject line")
    html: str = Field(description="Email HTML body")


email_agent = Agent(
    name="EmailAgent",
    instructions=EMAIL_INSTRUCTIONS,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="auto"),
    output_type=EmailOutput,
    tools=[send_email],  # the LLM can choose to call send_email or just return the JSON
)


In [27]:

# Writer agent — produces a structured Markdown report from search results
WRITER_INSTRUCTIONS = """
You are a medical writer. Write a concise, structured **Markdown report** about diabetes treatment
based on provided summarized search results. Structure:
- Title
- Key Takeaways (3–6 bullets)
- New/Updated Therapies (short bullets)
- Devices/Tech (CGM, pumps, closed-loop)
- Guidelines & Safety (ADA or others)
- References: list names of sources and year in brackets
Limit to ~400–600 words. Avoid speculation.
"""

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="auto"),
)


In [28]:

# Report container
class ReportData(BaseModel):
    markdown_report: str


In [29]:

# Helper pipeline functions (sync to match earlier Runner.run usage)

def plan_searches(query: str):
    """Use the planner agent to suggest distinct searches."""
    res = Runner.run(planner_agent, query)
    return res.final_output

def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a Markdown report from search results."""
    input_text = f"Original query: {query}\nSummarized search results: {search_results}"
    res = Runner.run(writer_agent, input_text)
    return res.final_output

def compose_and_send_email(markdown_report: str):
    """Use the email agent to generate subject/HTML and (optionally) send."""
    res = Runner.run(email_agent, markdown_report)
    # Try to send via the tool (if the model didn't already call it)
    try:
        if isinstance(res.final_output, dict) and {"subject", "html"}.issubset(res.final_output.keys()):
            send_status = send_email(res.final_output["subject"], res.final_output["html"])
        else:
            # If output typed, access attributes
            send_status = send_email(res.final_output.subject, res.final_output.html)
    except Exception as e:
        send_status = {"status": "error", "reason": str(e)}
    return res.final_output, send_status



### Example — End‑to‑End: plan → search → report → email

> This runs a small plan, searches with the **Search Agent**, writes a report, then composes & (optionally) sends an email.


In [30]:

import asyncio

async def run_end_to_end():
    # 1) Plan queries (async planner)
    topic = "Diabetes treatment 2025 updates"
    plan_res = await Runner.run(planner_agent, topic)
    plan = plan_res.final_output

    # Normalize searches whether Pydantic object or plain dict
    searches = getattr(plan, "searches", None)
    if searches is None and isinstance(plan, dict):
        searches = plan.get("searches", [])
    if searches is None:
        searches = []

    # 2) Run plan (short) — reuse Search Agent (await each search)
    summaries = []
    for item in searches[:2]:  # keep short for demo
        query = getattr(item, "query", item.get("query") if isinstance(item, dict) else None)
        if not query:
            continue
        sr = await Runner.run(search_agent, query)
        summaries.append(sr.final_output)

    # 3) Write a Markdown report from the summaries (call writer directly, await)
    input_text = f"Original query: {topic}\nSummarized search results: {summaries}"
    writer_res = await Runner.run(writer_agent, input_text)
    md_report = writer_res.final_output
    display(Markdown(md_report))

    # 4) Compose email (await) and send via tool
    email_res = await Runner.run(email_agent, md_report)
    # Support dict or typed output
    if isinstance(email_res.final_output, dict):
        subj = email_res.final_output.get("subject")
        html = email_res.final_output.get("html")
    else:
        subj = getattr(email_res.final_output, "subject", None)
        html = getattr(email_res.final_output, "html", None)

    send_status = {"status": "skipped", "reason": "missing email content"}
    if subj and html:
        try:
            send_status = send_email(subj, html)
        except Exception as e:
            send_status = {"status": "error", "reason": str(e)}

    print("Email compose/send status:", send_status)

await run_end_to_end()


# Diabetes Treatment Update (2025)

## Key Takeaways
- Advanced pharmacotherapy has expanded options for managing diabetes, especially for type 2 diabetes patients with additional health conditions.
- New therapies like oral GLP-1 receptor agonists and biosimilar insulins are enhancing treatment adherence and affordability.
- Continuous glucose monitor (CGM) technologies and closed-loop systems are transforming diabetes management, promoting better glycemic control.

## New/Updated Therapies
- **GLP-1 Receptor Agonists**: Expanded indications for cardiovascular risk reduction and renal protection; now recommended for patients with type 2 diabetes, chronic kidney disease, obesity, heart failure, and nonalcoholic fatty liver disease.
- **SGLT2 Inhibitors**: Recommended for patients with heart failure and chronic kidney disease with added cardiovascular benefits.
- **Biosimilar Insulin**: FDA approved insulin aspart-szjj, the first short-acting biosimilar insulin, introducing more affordable treatment options.

## Devices/Tech
- **Continuous Glucose Monitors (CGMs)**: The Dexcom G7 provides real-time glucose readings, integrated with the Omnipod 5 insulin pump, enhancing monitoring accuracy.
- **Insulin Pumps**: Medtronic's MiniMed Fit and Beta Bionics' Mint insulin pumps are in development, focusing on improved convenience and wearability.
- **Closed-Loop Systems**: Tandem Diabetes Care's Control-IQ+ system has shown significant improvements in glycemic control, with increased time in range and reduced HbA1c levels.

## Guidelines & Safety
- The **2025 American Diabetes Association (ADA) Standards of Care** promote personalized treatment strategies that prioritize safety and efficacy, particularly in high-risk patient populations, including those with cardiovascular and kidney diseases.
- Continuous assessment and integration of new treatment options are emphasized to optimize patient outcomes.

## References
- diabetesincontrol.com (2025)
- drugtopics.com (2025)
- en.wikipedia.org (2025)
- beyondtype1.org (2025)
- diatribe.org (2025)

Email compose/send status: {'status': 'error', 'reason': "'FunctionTool' object is not callable"}



## 7) Other Agents

Below are extra agents mirroring the style in your sample. They reuse the same `Agent(...)` pattern.


In [31]:

# SummarizerAgent — condense long text into bullets
SUMMARIZER_INSTRUCTIONS = """
Summarize the provided text into 5–8 crisp bullets highlighting the most important clinical or technical points.
Avoid redundancy. Keep each bullet single-line.
"""

summarizer_agent = Agent(
    name="SummarizerAgent",
    instructions=SUMMARIZER_INSTRUCTIONS,
    model="gpt-4o-mini",
)

# FormatterAgent — reformat raw notes into a clean report style
FORMATTER_INSTRUCTIONS = """
Reformat the raw notes into a clean, consistent Markdown brief with headings and bullets.
Do not change factual content; only organize and clarify.
"""

formatter_agent = Agent(
    name="FormatterAgent",
    instructions=FORMATTER_INSTRUCTIONS,
    model="gpt-4o-mini",
)

# NotifierAgent (stub) — could integrate with Slack via a tool (not implemented here)
NOTIFIER_INSTRUCTIONS = """
Draft a one-paragraph notification suitable for Slack or Teams summarizing a new diabetes-treatment update.
Keep it neutral and link-friendly. Return plain text only.
"""

notifier_agent = Agent(
    name="NotifierAgent",
    instructions=NOTIFIER_INSTRUCTIONS,
    model="gpt-4o-mini",
)
